### LangChain Vectorstore RAG Implementation
---
This notebook demonstrates a Retrieval-Augmented Generation (RAG) system using LangChain with local models via Ollama. The implementation follows a multi-step reasoning process:

1. **Setup**: Loads two Ollama models (phi4-mini for reasoning and Gemma3:1b for synthesis) to handle different parts of the process.

2. **Question Analysis**: Uses the reasoning model to break down complex questions into logical sub-steps that can be individually researched.

3. **Document Processing**: Loads a local text corpus about space exploration, splits it into manageable chunks, and creates vector embeddings using the nomic-embed-text model.

4. **Knowledge Retrieval**: For each identified reasoning step, performs a similarity search in the Chroma vectorstore to find the most relevant information from the knowledge base.

5. **Answer Synthesis**: Feeds the original question and all retrieved contextual information to the synthesis model, which generates a cohesive, factual response.

This approach enhances the quality of AI-generated answers by combining structured reasoning with targeted information retrieval from a domain-specific knowledge base, allowing for more accurate and contextually relevant responses than using an LLM alone.

#### To Do List

Add multiple documents to the doc store with the information needed to answer the question split between different documents
Add a PDF or other file type to make the search deal with different types of source data

In [1]:
# Install the required libraries from requirements.txt 
# pip install -r requirements.txt

In [2]:
try:
    # Import the built-in regular expressions module for pattern matching and text processing
    import re

    # Import the Ollama LLM class from the LangChain community package (often used for integrating local LLMs)
    from langchain_community.llms import Ollama

    # Import the PromptTemplate class used to define and structure prompts for LLMs
    from langchain.prompts import PromptTemplate

    # Import RunnableMap, a utility for composing and executing a sequence of runnable components
    from langchain_core.runnables import RunnableMap

    # Import Chroma vector store, used for storing and searching vector embeddings (RAG retrieval)
    from langchain.vectorstores import Chroma

    # Import Ollama-specific embeddings and LLM classes for use with LangChain
    from langchain_ollama import OllamaEmbeddings, OllamaLLM

    # Import TextLoader to load plain text documents from files for processing
    from langchain.document_loaders import TextLoader

    # Import CharacterTextSplitter to split large documents into smaller chunks based on character count
    from langchain.text_splitter import CharacterTextSplitter

except ImportError as e:
    print(f"Import error: {e}")
except Exception as e:
    print(f"Unexpected error during imports: {e}")


In [3]:
# --- Step 1: Ensure Ollama models are available ---
# Attempt to load the reasoning and synthesis LLM models from Ollama.
# If the models are not available, provide instructions to the user and exit.
try:
    reasoning_llm = OllamaLLM(model="phi4-mini")
    synthesis_llm = OllamaLLM(model="Gemma3:1b")
except Exception as e:
    print("❌ Failed to connect to Ollama or load model 'phi4-mini'.")
    print("💡 Make sure Ollama is running and the model is available:")
    print("    ollama run phi4-mini")
    print(f"Error details: {e}")
    exit(1)

In [4]:
# --- Step 2: Prompt to break down the question ---
# Define a prompt template to break down a question into logical steps.
# This uses the reasoning LLM to generate a step-by-step breakdown.
reasoning_prompt = PromptTemplate.from_template("""
You are a reasoning assistant. Break the following question into logical steps to help answer it:

Question: {question}

Step-by-step breakdown:
""")
step_chain = reasoning_prompt | reasoning_llm

In [ ]:
# --- Step 3: Set up vectorstore with Chroma ---
# Load a local text corpus using TextLoader.
loader = TextLoader("C:\Python\Agent-School\docs\Space.txt")  # Load your local corpus
documents = loader.load()

# Split the documents into smaller chunks for better processing.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# Set up embeddings using the Ollama 'nomic-embed-text' model.
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Create a vectorstore (Chroma) using the split documents and embeddings.
vectorstore = Chroma.from_documents(split_docs, embeddings)

# Here's example of how to persist the vectorstore to disk:
# You can install the SQLite VSCode extension to view the database.
# Use ctrl+shift+p to open the command palette and type "SQLite" to find the extension.
# Then choose SQLIte: Open Database and select the database file.
persist_directory = "./data/db/chroma"
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings, # openai embeddings
    persist_directory=persist_directory)

print(vectorstore._collection.count())

Created a chunk of size 505, which is longer than the specified 500


10


In [6]:
# --- Step 4: Prompt for final synthesis ---
# Define a prompt template for synthesizing a final answer.
# This uses the synthesis LLM to generate a complete and informative response.
# Remember - this is just a template, with {question} and {facts} as placeholders which are populated later.

synthesis_prompt = PromptTemplate.from_template("""
Based on the following question and information, write a complete, informative answer.

Question: {question}

Information:
{facts}

Answer:
""")
synthesis_chain = synthesis_prompt | synthesis_llm

In [7]:
# --- Step 5: Ask a question ---
# Define the question to be answered and invoke the reasoning chain to get step-by-step reasoning.
question = "Who was the U.S. president during the moon landing, and what was his policy on space exploration?"
steps_text = step_chain.invoke({"question": question})

# Print the reasoning steps generated by the LLM.
print("\n🧠 Reasoning steps:\n")
print(steps_text)


🧠 Reasoning steps:

Sure! Let's break down this task step by step.

1. Identify which event is being referred to as "the moon landing."
   - The Moon Landing refers specifically to Apollo 11's mission where Neil Armstrong first walked on the lunar surface in July 1969.
  
2. Determine who was President of the United States at that time (July-August 1969).
   - Richard Nixon served as U.S. President from January 20, 1969.

3. Find out what policies or initiatives related to space exploration were implemented by this president during his tenure around August-September 1969.
   - Although there isn't a well-documented specific "policy" directly linked with the Apollo 11 mission itself (it was primarily executed under President John F. Kennedy's administration), Richard Nixon continued many aspects of previous administrations' policies on space exploration, including NASA’s projects.

4. Summarize this information:
    - The U.S. president during the moon landing in July-August 1969 is id

In [8]:
# --- Step 6: Parse reasoning steps ---
# Extract individual reasoning steps from the generated text using regex.
step_lines = re.findall(r"\d+\.\s+(.*)", steps_text)
facts = []

In [9]:
# --- Step 7: Lookup each reasoning step with vectorstore ---
# For each reasoning step, perform a similarity search in the vectorstore.
for step in step_lines:
    print(f"\n🔍 Looking up: {step}")
    
    # Retrieve the most relevant document from the vectorstore.
    docs = vectorstore.similarity_search(step, k=1)
    result = docs[0].page_content if docs else "No relevant info found in local knowledge base."
    
    # Append the result to the list of facts.
    facts.append(f"- {step.strip()}: {result}")

# Combine all retrieved facts into a single string.
combined_facts = "\n".join(facts)

# Print the retrieved facts.
print("\n📚 Retrieved facts from Vectorstore:\n")
print(combined_facts)


🔍 Looking up: Identify which event is being referred to as "the moon landing."

🔍 Looking up: 2. Determine who was President of the United States at that time (July-August 1969).

🔍 Looking up: 3. Find out what policies or initiatives related to space exploration were implemented by this president during his tenure around August-September 1969.

🔍 Looking up: Summarize this information:

📚 Retrieved facts from Vectorstore:

- Identify which event is being referred to as "the moon landing.": ### Nixon's Role in the Moon Landing
When Nixon took office in January 1969, the Apollo program was already well underway, a result of Kennedy's ambitious goal to land a man on the moon before the end of the decade. Nixon inherited the program at its climax and ensured its success was celebrated as a national achievement. He famously spoke to astronauts Neil Armstrong and Buzz Aldrin during their time on the lunar surface, calling it "the greatest week in the history of the world since the Creation

In [10]:
# --- Step 8: Summarize using second LLM ---
# Use the synthesis chain to generate a final answer based on the question and retrieved facts.
final_answer = synthesis_chain.invoke({
    "question": question,
    "facts": combined_facts
})

# Print the final synthesized answer.
print("\n✅ Final synthesized answer:\n")
print(final_answer)


✅ Final synthesized answer:

During the historic Apollo 11 moon landing in July 1969, the U.S. president of the United States was Richard Nixon. His policy on space exploration was more pragmatic than Kennedy’s, focusing on balancing space exploration with other national priorities. Under his administration, NASA’s budget began to decline, and the Apollo program was scaled back after Apollo 17 in 1972.
